In [1]:
import pandas as pd
from sklearn.cluster import KMeans

In [2]:
movies_data = pd.read_csv('movies.csv')
# print(movies_data)

ratings_data = pd.read_csv('ratings.csv')
# print(ratings_data)

       movieId                                title  \
0            1                     Toy Story (1995)   
1            2                       Jumanji (1995)   
2            3              Grumpier Old Men (1995)   
3            4             Waiting to Exhale (1995)   
4            5   Father of the Bride Part II (1995)   
...        ...                                  ...   
10324   146684        Cosmic Scrat-tastrophe (2015)   
10325   146878           Le Grand Restaurant (1966)   
10326   148238       A Very Murray Christmas (2015)   
10327   148626                 The Big Short (2015)   
10328   149532  Marco Polo: One Hundred Eyes (2015)   

                                            genres  
0      Adventure|Animation|Children|Comedy|Fantasy  
1                       Adventure|Children|Fantasy  
2                                   Comedy|Romance  
3                             Comedy|Drama|Romance  
4                                           Comedy  
...                  

In [3]:
def load_data(movies_data, ratings_data):
    movie_ratings = pd.merge(movies_data, ratings_data, on='movieId')
    return movie_ratings


In [4]:
def cluster_users(movie_ratings, n_clusters=3):
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    movie_ratings['user_cluster'] = kmeans.fit_predict(movie_ratings['userId'].values.reshape(-1, 1))
    return movie_ratings

In [5]:
def get_user_cluster(target_user, movie_ratings):
    return movie_ratings[movie_ratings['userId'] == target_user]['user_cluster'].values[0]

In [6]:
def get_similar_users(target_user_cluster, movie_ratings):
    similar_users = movie_ratings[movie_ratings['user_cluster'] == target_user_cluster]['userId'].unique()
    return similar_users

In [7]:
def get_similar_user_data(similar_users, movie_ratings):
    similar_user_data = movie_ratings[movie_ratings['userId'].isin(similar_users)]
    return similar_user_data

In [8]:
def calculate_average_rating(similar_user_data):
    avg_rating = similar_user_data.groupby('title')['rating'].mean()
    return avg_rating


In [9]:
def get_top_recommendations(target_user, num_recommendations,min_users_threshold ,avg_rating, movie_ratings):
    target_user_movies = movie_ratings[movie_ratings['userId'] == target_user]['title']
    top_movies = avg_rating.sort_values(ascending=False)
    top_movies = top_movies[~top_movies.index.isin(target_user_movies)]
    recommended_movies = top_movies.head(num_recommendations)
    return recommended_movies